In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Get to the folder we are at
FOLDERNAME = 'Colab\ Notebooks/Kaggle'
%cd drive/MyDrive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Kaggle


In [2]:
%%capture
!pip install -q ipython-autotime
%load_ext autotime

!pip install -q 'monai[nibabel, matplotlib]'
!python -c "import monai; import nibabel"
!pip install torcheval

time: 26.7 s (started: 2024-02-07 15:29:02 +00:00)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
from functools import partial, partialmethod
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os
from PIL import Image
from zipfile import ZipFile

from monai.transforms import (
    Compose,
    CropForegroundd,
    EnsureTyped,
    LoadImaged,
    NormalizeIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    SpatialPadd,
    ThresholdIntensityd)

from monai.data import (
    DataLoader,
    Dataset,
    ThreadDataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
    pad_list_data_collate)

time: 12.2 s (started: 2024-02-07 15:29:29 +00:00)


In [4]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
device

device(type='cuda')

time: 78 ms (started: 2024-02-07 15:29:41 +00:00)


In [5]:
test_transforms = Compose(
    [
        LoadImaged(keys=["image"], simple_keys=True, image_only=False, ensure_channel_first=True, reader="NibabelReader"),
        Spacingd(keys=["image"], pixdim=(0.684, 0.684, 5.0), mode=("nearest")),
        CropForegroundd(keys=["image"], source_key="image", allow_smaller=False),
        SpatialPadd(keys=["image"], spatial_size=[160, 160, 32]),
        EnsureTyped(keys=["image"], device=device, track_meta=True)
    ])

time: 22.9 ms (started: 2024-02-07 15:29:41 +00:00)


In [6]:
def unzip(zip_path, save_path):
  with ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting"):
      file_path = os.path.join(save_path, file)
      zip_ref.extract(file, save_path)

time: 7.37 ms (started: 2024-02-07 15:29:41 +00:00)


In [7]:
save_path = "Data/Inference_test"
test_zip_path = f"{save_path}/ResNext3D_classification_inference.zip"

time: 365 µs (started: 2024-02-07 15:29:41 +00:00)


In [8]:
unzip(test_zip_path, save_path)

Extracting: 100%|██████████| 177/177 [00:01<00:00, 102.70it/s]

time: 3.32 s (started: 2024-02-07 15:29:42 +00:00)


In [9]:
split_json = "Data/Inference_test/ResNext3D_classification_inference.json"

test_files = load_decathlon_datalist(split_json, base_dir=save_path, is_segmentation=False, data_list_key="test")
test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_num=12, cache_rate=1.0, num_workers=8)

Loading dataset: 100%|██████████| 12/12 [00:01<00:00, 11.20it/s]

time: 1.78 s (started: 2024-02-07 15:29:45 +00:00)


In [10]:
batch = 32
test_loader = ThreadDataLoader(test_ds, num_workers=0, batch_size=batch)

time: 3.2 ms (started: 2024-02-07 15:29:47 +00:00)


In [11]:
from ResNeXt3D.resnext3d import generate_model

# model_depth in [50, 101, 152, 200]
model = generate_model(152, n_input_channels=1, n_classes=1).to(device)
torch.backends.cudnn.benchmark = True

model_name = "ResNext3D_model_bymask_v4_best"
checkpoint = torch.load(f"{model_name}.pth")  # map_location=torch.device('cpu')
model.load_state_dict(checkpoint['model_state_dict'])
tqdm = partial(tqdm, position=0, leave=True)

time: 16 s (started: 2024-02-07 15:29:47 +00:00)


In [12]:
def inference(test_loader, test_iterations):
  model.eval()
  test_iterator = tqdm(test_loader, desc="Testing (X / X Steps)", dynamic_ncols=True)

  res = []
  with torch.no_grad():
    for step, data in enumerate(test_iterator):
      inputs, file_names = data["image"].to(device), data['image_meta_dict']["filename_or_obj"]
      with torch.cuda.amp.autocast():
        preds = (torch.sigmoid(model(inputs)) >= 0.5).float() * 1.0
        # preds = torch.sigmoid(model(inputs))

      file_names = [file.split('/')[-1].replace("_cropped.nii.gz", "") for file in file_names]
      preds = [int(pred) for pred in decollate_batch(preds)]
      res.extend([file_name, pred_res] for file_name, pred_res in zip(file_names, preds))

      # update procedure
      test_iterator.set_description(f"Testing ({step + 1} / {test_iterations} Steps)", refresh=True)

  df = pd.DataFrame(res, columns=["scan_id", "pred"])
  df.to_csv(f"{save_path}/ResNext3D_classification_inference_result.csv", index=False)

time: 830 µs (started: 2024-02-07 15:30:03 +00:00)


In [13]:
test_iterations = math.ceil(len(test_ds) / batch)
inference(test_loader, test_iterations)

Testing (6 / 6 Steps): 100%|██████████| 6/6 [00:17<00:00,  2.95s/it]

time: 17.7 s (started: 2024-02-07 15:30:03 +00:00)


time: 17.7 s (started: 2024-02-07 15:30:03 +00:00)
